In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%aiida
import urllib.parse as urlparse
import ipywidgets as ipw
from IPython.display import display, clear_output
from aiidalab_widgets_base import viewer
import nglview
import numpy as np
import bokeh.plotting as bpl
import bokeh.models as bmd
from bokeh.layouts import row
from bokeh.io import output_notebook, show

_ColormakerRegistry()

In [3]:
def plot(x, y, x_dev=None, y_dev=None, title=None, xlabel=None, ylabel=None):
    x = np.array(x)
    y = np.array(y)

    output_notebook(hide_banner=True)
    fig = bpl.figure(
        toolbar_location=None,
        title=title,
        x_axis_label=xlabel,
        y_axis_label=ylabel,
        tools=['tap', 'zoom_in', 'zoom_out', 'pan'], 
        output_backend='webgl',
    )

    source = bmd.ColumnDataSource(data=dict(x=x, y=y))
    
    fig.line('x', 'y', source=source)
    fig.circle('x', 'y', size=10, fill_color="white", source=source)
    
    if x_dev:
        x_dev = np.array(x_dev)
        source_dev = bmd.ColumnDataSource(data=dict(base=y, lower=x-x_dev, upper=x+x_dev))
        fig.add_layout(bmd.Whisker(source=source_dev, base="base", upper="upper", lower="lower", dimension='width'))
    
    if y_dev:
        y_dev = np.array(y_dev)
        source_dev = bmd.ColumnDataSource(data=dict(base=x, lower=y-y_dev, upper=y+y_dev))
        fig.add_layout(bmd.Whisker(source=source_dev, base="base", upper="upper", lower="lower"))
        
    return fig

In [4]:
url = urlparse.urlsplit(jupyter_notebook_url)
uuid = urlparse.parse_qs(url.query)['uuid'][0]
workcalc = load_node(uuid=str(uuid))
ism = workcalc.outputs.output_parameters['isotherm']

isotherm = np.array([ism['pressure'], ism['loading_absolute_average'], ism['loading_absolute_dev']]).transpose()



enthalpy = np.array([ism['enthalpy_of_adsorption_average'], ism['enthalpy_of_adsorption_dev']]).transpose()

# Cleaning up.
to_delete = []
for i, p in enumerate(enthalpy):
    if None in p:
        to_delete.append(i)

if to_delete:
    enthalpy = np.delete(enthalpy, *to_delete, axis=0)
    isotherm = np.delete(isotherm, *to_delete, axis=0)

enth_av, enth_dev = zip(*enthalpy)
ism_p, ism_load, ism_dev = zip(*isotherm)


molecule_converter = {
    "co2": "CO2",
    "ch4": "CH4",
    "n2" : "N2",
    "h2o": "H2O",
    "h2" : "H2",
    "o2" : "O2",
}

# Detailed Report

## Structure

In [6]:
display(viewer(workcalc.inputs.structure))

StructureDataViewer(children=(HBox(children=(VBox(children=(NGLWidget(), ToggleButtons(description='Camera typ…

## Isotherm and Enthalpy plots

In [7]:
if workcalc.outputs.output_parameters['is_porous']:
    isotherm_plot = plot(x=ism_p, y=ism_load, y_dev=ism_dev, xlabel="Pressure [{}]".format(ism['pressure_unit']),
                         ylabel="{} loading [{}]".format(molecule_converter[workcalc.inputs['molecule'].value],
                                         ism['loading_absolute_unit']))
    isotherm_plot.height = 500
    isotherm_plot.width = 600


    enthalpy_plot = plot(x=enth_av, y=ism_load, x_dev=enth_dev,
                         xlabel="Enthalpy of adsorption [{}]".format(ism['enthalpy_of_adsorption_unit']),
    ylabel="{} loading [{}]".format(molecule_converter[workcalc.inputs['molecule'].value],
                                         ism['loading_absolute_unit']))

    enthalpy_plot.height = 500
    enthalpy_plot.width = 400

    enthalpy_plot.x_range = bmd.Range1d(min(enth_av)-2.0, max(enth_av)+2.0)

    show(row(isotherm_plot, enthalpy_plot))
else:
    print("The material is not porous. No isotherm produced.")

## Computed properties

In [8]:
display(viewer(workcalc.outputs.output_parameters))

DictViewer(value='\n        <style>\n            .df { border: none; }\n            .df tbody tr:nth-child(odd…